In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Kinematic computations 

In [3]:
m = compile(robot)
kcache = new_kinematics_cache(m)  

rh_mftip_frame_id = get_compiled_frameID(m, "rh_ffmiddle")
mftip_transform = get_transform(kcache, rh_mftip_frame_id)
display(mftip_transform.origin)

3-element SVector{3, Float64} with indices SOneTo(3):
  0.033
 -0.01
  0.38701

### Hand Visualization

In [4]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.045, -0.01, 0.38701]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.045
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.38701
 0.0  0.0  0.0   1.0

In [5]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_ffmiddle")
display_frame(m, ls, "world")
display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [ ]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), frames(robot))
show(IOContext(stdout, :limit => false),  MIME("text/plain"), keys(joints(robot)))

33-element Vector{String}:
 "world"
 "rh_forearm"
 "rh_wrist"
 "rh_palm"
 "rh_manipulator"
 "rh_imu"
 "rh_ffknuckle"
 "rh_ffproximal"
 "rh_ffmiddle"
 "rh_ffdistal"
 "rh_fftip"
 "rh_mfknuckle"
 "rh_mfproximal"
 "rh_mfmiddle"
 "rh_mfdistal"
 "rh_mftip"
 "rh_rfknuckle"
 "rh_rfproximal"
 "rh_rfmiddle"
 "rh_rfdistal"
 "rh_rftip"
 "rh_lfmetacarpal"
 "rh_lfknuckle"
 "rh_lfproximal"
 "rh_lfmiddle"
 "rh_lfdistal"
 "rh_lftip"
 "rh_thbase"
 "rh_thproximal"
 "rh_thhub"
 "rh_thmiddle"
 "rh_thdistal"
 "rh_thtip"

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [6]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Addition of the multiples springs/dampers

In [ ]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

m = compile(robot)
kcache = new_kinematics_cache(m)  

cart_init_pos = SVector(0.011, -0.01, 0.442)

#Linking the four fingers to the same point, with non-zero equilibrium springs

K = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
D = SMatrix{3, 3}(10., 0., 0., 0., 10.0, 0., 0., 0., 10.)

#lightly constraining some joints to avoid unwanted motions
add_component!(vms, LinearSpring(10.0, ".robot.rh_LFJ5_coord"); id = "lf j5 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_FFJ4_coord"); id = "ff j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_MFJ4_coord"); id = "mf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_RFJ4_coord"); id = "rf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_LFJ4_coord"); id = "lf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_WRJ1_coord"); id = "wr j1 angular spring")
add_component!(vms, LinearSpring(0.001, ".robot.rh_FFJ3_coord"); id = "ff j3 angular spring")


# THUMB HANDLING 
add_coordinate!(vms, FrameOrigin(".robot.rh_thtip"); id ="th position")
add_coordinate!(vms, FramePoint(".robot.rh_ffmiddle", SVector(0.012,0.0,0.0)); id= "th target position")
add_coordinate!(vms, CoordDifference("th position", "th target position"); id = "th target dist")
add_coordinate!(vms, CoordNorm("th target dist"); id="th target norm")
add_coordinate!(vms, ConstCoord(0.07); id="th spring length")
add_coordinate!(vms, CoordDifference("th target norm", "th spring length"); id = "th position error")

add_component!(vms, LinearSpring(100.0, "th position error"); id="th position spring")
add_component!(vms, LinearDamper(100.0, "th position error"); id="th position damper")

add_coordinate!(vms, ConstCoord(1.57); id = "ff j2 angle target")
add_coordinate!(vms, CoordDifference(".robot.rh_FFJ2_coord", "ff j2 angle target") ; id ="ff j2 angle error")
add_component!(vms, LinearSpring(0.0, "ff j2 angle error"); id = "ff j2 spring")

# add_coordinate!(vms, ConstCoord(1.00); id="th j5 target angle")
# add_coordinate!(vms, JointSubspace(".robot.rh_THJ5"); id= "th j5 angle")
# add_coordinate!(vms, CoordDifference("th j5 target angle", "th j5 angle"); id="th j5 angle error")

# add_component!(vms, LinearSpring(100.0, "th j5 angle error"); id="th j5 angular spring")
# add_component!(vms, LinearDamper(10.0, "th j5 angle error"); id="th j5 angular damper")


"ff j2 spring"

## Simulating the Robot

### Setting Up the Simulation

In [8]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

"""
Update the vms based on the "coord", which is a coordinate varying between 
0 and 1, 0 being the starting position of the lateral pinch motion while 1 being 
the end
"""
function update_lateral_pinch_coord(args, cache, coord)
    #target_rail_id, th_spring_length_id, th_j5_target_angle_id = args
    th_spring_length_id, th_target_pos_id, ffmiddle_frame_id, ff_j2_spring_id = args

    #update the length of the spring between the thumb and the ff finger

    # First phase : the thumb converges to the top of the phalanx
    if coord < 0.6
        length_max = 0.07
        length_min = 0.0
        length_value = length_max - (length_max - length_min)*(coord/0.6) 
        cache[th_spring_length_id] = remake(cache[th_spring_length_id] ; coord_data = ConstCoord(length_value))

    # phase 2 : a force is applied to go "into" the phalanx
    elseif coord > 0.8
        cache[th_target_pos_id] = remake(cache[th_target_pos_id]; coord_data = FramePoint(ffmiddle_frame_id, SVector(0.,0.,0.)))
    end

    #update the spring of the second phalanx
    j2_activation_coord = 0.3
    if coord > j2_activation_coord
        stiff_max = 0.1
        stiff_min = 0.0
        stiff_value = stiff_min + (stiff_max - stiff_min)*((coord - j2_activation_coord)/(1.0 - j2_activation_coord))
        cache[ff_j2_spring_id] = remake(cache[ff_j2_spring_id] ; stiffness = stiff_value)
    end 

    # #update the angle of the thumb
    # angle_max = 1.0
    # angle_min = 0.0
    # angle_value = angle_max - (angle_max - angle_min)*coord
    # cache[th_j5_target_angle_id] = remake(cache[th_j5_target_angle_id] ; coord_data = ConstCoord(angle_value))

    nothing
end

function f_setup(cache) 
    return  (get_compiled_coordID(cache, "th spring length"), get_compiled_coordID(cache, "th target position"), 
            get_compiled_frameID(cache, ".robot.rh_ffmiddle"), get_compiled_componentID(cache, "ff j2 spring"))#, get_compiled_coordID(cache, "th j5 target angle"))
end

function f_control(cache, t, args, extra)

    t_start = 2.
    t_end = 8. #we want to reach the end position at t = t_end 
    
    if t >= t_start && t <= t_end 
        coord_value = (t - t_start) /(t_end -t_start)
        update_lateral_pinch_coord(args, cache, coord_value)
    end
end

tspan = (0., 10.)
vms_compiled = compile(vms)

q_init = generate_q_init(vms_compiled; mf=true, rf=true, lf=true)
q = (q_init, zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with lateral pinch"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-5, reltol=1e-5);

[ Info: Simulating shadow robot with lateral pinch
ODE   0%|█                                              |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:08:05
ODE   4%|██                                             |  ETA: 0:04:23
ODE   6%|███                                            |  ETA: 0:03:08
ODE   8%|████                                           |  ETA: 0:02:29
ODE  10%|█████                                          |  ETA: 0:02:05
ODE  12%|██████                                         |  ETA: 0:01:49
ODE  14%|███████                                        |  ETA: 0:01:40
ODE  16%|████████                                       |  ETA: 0:01:31
ODE  18%|█████████                                      |  ETA: 0:01:24
ODE  20%|██████████                                     |  ETA: 0:01:20
ODE  21%|███████████                                    |  ETA: 0:01:18
ODE  23%|███████████                                    |  ETA: 0:01:17
ODE  24%|████████

### Visualizing the Results

In [10]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_lateral_pinch.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);